In [9]:
%load_ext autoreload 
%autoreload 2
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from causaldag import UndirectedGraph
import itertools
from sklearn import preprocessing
import pickle
from ut_gsp_stable import unknown_target_igsp_stability_selection
from gsp import gsp_stability_selection
from gspo_stable_all import gspo_stability_selection_both
from gspo_stable_bidi_only import gspo_stability_selection_bidi_only
from gspo_stable_dir_only import gspo_stability_selection_dir_only
from graphviz_plotting import plot_graphviz_style_graph
import os
from IPython.display import Image

%matplotlib inline
import numpy as np
from causaldag import unknown_target_igsp
import causaldag as cd
import random
from causaldag import gauss_invariance_suffstat, gauss_invariance_test, MemoizedInvarianceTester, gspo, gsp
from causaldag import partial_correlation_suffstat, partial_correlation_test, MemoizedCI_Tester

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Functions
def choose_stable_variables(stability_scores, bootstrap_threshold=0.5):
    """Returns adjacency matrix corresponding to edges with stability scores above threshold."""
    return (stability_scores.max(axis=0) > bootstrap_threshold).astype('float')


def get_directed_and_undirected_edges(adjacency_matrix):
    """
    Given an adjacency matrix, which contains both directed and undirected edges,
    this function returns two adjancy matrices containing directed and undirected edges separately.
    Useful for plotting the difference causal graph.
    Parameters
    ----------
    adjacency_matrix: array, shape  = [num_nodes, num_nodes]
        Adjacency matrix representing partially directed acyclic graph,
        which containts both undirected and directed edges. 
        Each entry should be either 0 or 1, representing absence or presence of an edge, respectively.
    Returns
    -------
    adjacency_matrix_directed: array, shape  = [num_nodes, num_nodes]
        Adjacency matrix containing only directed edges.
    adjacency_matrix_undirected: array, shape  = [num_nodes, num_nodes]
        Adjacency matrix containing only undirected edges.
    """

    adjacency_matrix = adjacency_matrix.astype('float')
    adjacency_matrix_sym = adjacency_matrix + adjacency_matrix.T
    adjacency_matrix_undirected = (adjacency_matrix_sym == 2).astype('float')
    adjacency_matrix_directed = (adjacency_matrix_sym == 1).astype('float')
    adjacency_matrix_directed[adjacency_matrix_directed == 1] = adjacency_matrix[adjacency_matrix_directed==1]
    return adjacency_matrix_directed, adjacency_matrix_undirected



In [11]:
path_in = '/home/gridsan/jyun/dci'
df_original = pd.read_csv('/home/gridsan/jyun/dci/bd21d2.csv', index_col=0)
df_original.shape
df_original.shape[0]
path_in = '/home/gridsan/jyun/dci'
df_original1 = pd.read_csv('/home/gridsan/jyun/dci/bd21d1.csv', index_col=0)
df_original1.shape[0]

df_original2 = pd.read_csv('/home/gridsan/jyun/dci/bd21d4.csv', index_col=0)
df_original2.shape
df_original3 = pd.read_csv('/home/gridsan/jyun/dci/bd21d3.csv', index_col=0)

df_original3.shape
df_original1 = pd.concat([df_original1, df_original2,df_original3])
#df_original1 = pd.concat([df_original1, df_original2])
df_original1.shape
df_original.shape
df_original1=df_original1.iloc[:, :]
df_original=df_original.iloc[: , :]
df_original.shape

#df_original=df_original.iloc[:45 , :49]
#df_original=pd.concat([df_original.iloc[:45, 0], df_original.iloc[:45, 15:49]], axis=1)
#df_original=df_original.iloc[:46 , :49]
X=df_original
Y=df_original1
nsamples, nnodes = df_original.shape
nsamples
nnodes
nodes = set(range(nnodes))
nodes
alpha_grid = [0.0001, 0.001, 0.01, 0.05,0.1]
num_targets = 1
num_settings = 1
nodes = set(range(nnodes))
targets_list = [random.sample(nodes, num_targets) for _ in range(num_settings)]
setting_list = [dict(known_interventions=[]) for _ in targets_list]
print(setting_list)
alpha2=0.0001
X=X.to_numpy()
Y=Y.to_numpy()
df_original.transpose()

[{'known_interventions': []}]


,X29,X175,X212,X55,X163,X63,X56,X62,X231,X190,...,X228,X12,X11,X135,X157,X64,X116,X154,X109,X52
Bradi1g13760.2,5.166412,3.360318,5.442194,4.198412,3.145728,4.995799,2.394483,4.443975,4.183903,4.371803,...,4.135745,4.073664,4.746518,4.026916,3.939269,5.792740,4.541823,3.611585,4.486711,1.213531
Bradi1g32445.1,1.629822,0.825763,2.627647,1.060002,0.000000,1.347863,0.000000,1.689246,0.000000,0.922665,...,0.905009,1.955933,0.000000,1.065538,0.000000,1.950480,1.691220,0.849341,1.815573,2.309507
Bradi1g51245.1,7.555592,6.372989,7.434476,6.265764,5.421840,6.820661,6.330568,7.063409,7.185453,7.404080,...,6.816179,6.980318,7.145889,6.711912,6.840722,7.634355,7.284812,5.708943,7.782929,4.907804
Bradi1g51770.1,5.221450,4.106175,6.116412,4.504537,2.640052,5.416988,4.877052,5.251013,5.200918,6.334668,...,5.588505,4.498092,5.633640,4.444008,4.659266,6.163313,5.438181,4.646759,5.999996,0.730768
Bradi1g58560.1,2.864779,2.032062,3.044786,1.664423,0.000000,2.991693,2.394483,2.940100,1.281710,3.554265,...,2.166787,1.955933,2.823932,0.000000,3.296442,3.845003,2.119006,2.538555,3.295720,1.863181
Bradi1g75740.1,6.201985,5.363472,5.957126,5.606949,4.958511,5.700342,5.852294,5.949849,5.309041,6.235266,...,5.018457,6.071084,5.780519,5.517621,5.460132,5.689734,5.826102,5.569775,5.999996,4.508698
Bradi2g07310.3,4.587498,3.360318,4.135580,2.908737,3.145728,4.685131,4.281760,4.584388,5.200918,4.800559,...,4.062295,4.750447,4.825991,4.878564,4.527461,5.220732,4.735355,3.172613,4.640386,5.141399
Bradi2g15940.1,4.888900,3.739801,5.219602,3.693128,3.519426,4.848811,3.925330,4.938525,4.065570,4.654296,...,4.717217,4.825625,4.573094,4.208456,4.780031,5.865439,4.541823,4.020759,4.965319,2.925093
Bradi2g18510.1,6.268830,5.128821,6.526241,5.362560,5.233995,6.217309,5.580047,6.000864,5.966814,6.033074,...,5.760911,5.642539,5.839161,5.617352,5.564704,6.410085,5.854172,5.468929,5.683693,4.015160
Bradi2g20660.3,5.300259,4.169476,4.956289,4.756939,4.061830,4.685131,4.730494,4.866999,4.662866,5.237264,...,4.717217,5.683534,5.106220,5.551637,4.456749,5.426951,5.545369,4.280852,5.130104,3.045224


In [12]:
stability_scores = unknown_target_igsp_stability_selection(X, Y, alpha_grid, alpha2, nodes, setting_list, n_samples=df_original.shape[0],n_bootstrap_iterations=1000, n_jobs=48, verbose=False,sample_fraction=0.85)

0.0001
(46, 30)
0.001
(46, 30)
0.01
(46, 30)
0.05
(46, 30)
0.1
(46, 30)


In [13]:
np.savetxt("/home/gridsan/jyun/dci/adjacency_matrix_bd21d12.csv",pd.DataFrame(stability_scores.max(axis=0)), delimiter=",")